In [1]:
# network/comp. bio. packages
import networkx as nx
import cobra
from cobra.io import read_sbml_model
import netwulf
from netwulf import visualize

# helper packages
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import random

%matplotlib inline

In [2]:
# iterate over model.reactions and model.metabolites to construct bipartite directed network
def makeNetworkFromSBML(model):
    # directed grah
    G = nx.DiGraph()
    
    # metabolite nodes and reaction nodes
    nodes_m = {}
    nodes_r = {}

    # iterate through metabolites in model and add to nodes_m dict
    for metabolite in model.metabolites:
        nodes_m[metabolite.id] = metabolite.name
        
    # iterate through reactions in model and add to nodes_r dict
    for reaction in model.reactions:
        nodes_r[reaction.id] = reaction.name
    
    # add nodes from met/rxn dicts with bipartite attribute
    # add nodes with metabolite formula/enzyme name as attribute for id
    G.add_nodes_from([(m_id, {'name': name}) for (m_id, name) in nodes_m.items()], bipartite=0)
        ''' same as...
            for m_id, name in nodes_m.items():
                G.add_node(m_id, name = name, biartite = 0)'''
    
    G.add_nodes_from([(r_id, {'name': name}) for (r_id, name) in nodes_r.items()], bipartite=1)
    
    edges = []
    for reaction in model.reactions:
        # get products and reactants
        products  = reaction.products
        reactants = reaction.reactants
        enzyme = reaction.name
        rid = reaction.id
        
        for p in products:
            edges.append((rid, p.id))
            if reaction.reversibility:
                edges.append((p.id, rid))
                
        for r in reactants:
            edges.append((r.id, rid))
            if reaction.reversibility:
                edges.append((rid, r.id))
            
        
    G.add_edges_from(edges)
    return G


In [3]:
# create list of metabolic networks, one for each genome
networks = {}
models = {}
sbml_dir = 'sbml_files'
for sbml_file in os.listdir(sbml_dir):
    # print(sbml_file)
    
    # ignore JCVI for now...
    if sbml_file != 'JCVI.sbml':
        model = read_sbml_model(os.path.join(sbml_dir, sbml_file))
        models[sbml_file] = model
        networks[sbml_file] = makeNetworkFromSBML(model)

### Create JCVI model 

In [8]:
JCVI_model = read_sbml_model('sbml_files/JCVI.sbml')

In [12]:
G = makeNetworkFromSBML(JCVI_model)

In [10]:
data = visualize(G)

In [15]:
degree_list = sorted([d for n, d in G.degree()], reverse = True)
print(degree_list)
print(len(degree_list))


[217, 145, 57, 45, 42, 39, 38, 37, 35, 34, 30, 30, 28, 28, 28, 28, 26, 25, 24, 22, 20, 20, 20, 20, 20, 20, 20, 20, 18, 18, 18, 18, 18, 18, 18, 16, 16, 16, 16, 16, 15, 15, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 13, 13, 13, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 11, 11, 11, 11, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 

### collapse down to enzymes

In [46]:
mets, rxns = nx.bipartite.sets(G)

G_rxns = nx.bipartite.weighted_projected_graph(G, rxns, ratio = True)


In [52]:
for i in G_rxns.edges():
    print(G_rxns.edges[i]['weight'])

0.0033003300330033004
0.0033003300330033004
0.006600660066006601
0.0033003300330033004
0.006600660066006601
0.0033003300330033004
0.0033003300330033004
0.006600660066006601
0.0033003300330033004
0.0033003300330033004
0.0033003300330033004
0.0033003300330033004
0.0033003300330033004
0.0033003300330033004
0.006600660066006601
0.0033003300330033004
0.0033003300330033004
0.006600660066006601
0.0033003300330033004
0.0033003300330033004
0.006600660066006601
0.0033003300330033004
0.0033003300330033004
0.0033003300330033004
0.0033003300330033004
0.0033003300330033004
0.0033003300330033004
0.0033003300330033004
0.0033003300330033004
0.0033003300330033004
0.0033003300330033004
0.0033003300330033004
0.0033003300330033004
0.0033003300330033004
0.0033003300330033004
0.006600660066006601
0.0033003300330033004
0.0033003300330033004
0.006600660066006601
0.0033003300330033004
0.0033003300330033004
0.0033003300330033004
0.0033003300330033004
0.0033003300330033004
0.0033003300330033004
0.0033003300330033

In [47]:
data = visualize(G_rxns)

In [36]:
JCVI_model.reactions[2]

Reaction identifier,rxn15626_c
Name,R06482
Memory address,0x01a746f01820
Stoichiometry,"9.0 cpd00005_c + 21.0 cpd00067_c + 7.0 cpd00070_c + cpd00481_c + 5.0 cpd01675_c <=> 5.0 cpd00001_c + 9.0 cpd00006_c + 13.0 cpd00010_c + 12.0 cpd00011_c + cpd08750_c 9.0 Nadph + 21.0 H+ + 7.0 Malonyl-Coa + Isobutyryl-Coa + 5.0 Methylmalonyl-Coa <=> 5.0 H2O + 9.0 Nadp + 13.0 Coa + 12.0 Co2 + 6,8A-Seco-6,8A-Deoxy-5-Oxoavermectin ''2B'' Aglycone"
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [42]:
r1 = JCVI_model.reactions[1]

print(r1.reactants)
print(r1.products)

print(r1.reaction)

[<Metabolite cpd00115_c at 0x1a748483d00>]
[<Metabolite cpd00012_c at 0x1a7484834f0>]
cpd00115_c <-- cpd00012_c


In [ ]:
def getEnzymeDict(enzymes, graph):
    enzyme_dict = {}
    
    for e in enzymes:
        
        # accumulate products of enzyme reaction
        products = set()
        